In [ ]:
%matplotlib inline

In [ ]:
# import libraries 
import os
import numpy as np
import matplotlib.pyplot as plt
import mne

### Load the data 

Given that the data is in .fif format (MEG data obtained from a NEUROMAG scanner), the ```mne.io.read_raw_fif()``` function will be used 

The ```mne.io.read_raw_*``` functions do not load the data into memory automatically, but some operations such as filtering require that data be copied into RAM. To do that we can either use the ```preload=True``` argument or we can later load the raw in RAM using the ```load_data()``` method.

In [ ]:
sample_data_folder = '/Users/christinadelta/datasets/eeg_testing_data'
sample_data_raw_file = os.path.join(sample_data_folder, 'MEG', 'sample',
                                    'sample_audvis_raw.fif')
raw = mne.io.read_raw_fif(sample_data_raw_file)

raw.crop(tmax=60).load_data() # we'll use the 60 sec of the data for now 

We load the raw data in a **Raw object**. There are several ways to look at the data, but let's use the **Info object**

In [ ]:
print(raw)

In [ ]:
# or a simple way to look at channel names by using an attribute 
raw.ch_names

We can look at more detail information:

In [ ]:
data_info = mne.io.read_info(sample_data_raw_file)

Get more specific info from the the **data_info** object

In [ ]:
print(data_info.keys())

We can also extract data from specific keys of the **data_info** dict

In [ ]:
print(data_info['nchan'])
print() # insert blank line
print(data_info['chs'][0].keys())

### Get more info about the channels 

We can use the info obejct to obtain subsets of the channels. To work with channels, we can use two different functions: ```mne.pick_channels()``` or ```mne.pick_types```. ```mne.pick_channels()``` manually picks all the channels thus we should specify what to include or exclude:

In [ ]:
print(mne.pick_channels(data_info['ch_names'], include = ['MEG 0312', 'EEG 005']))

In [ ]:
# or we can use:
print(mne.pick_channels(data_info['ch_names'], include = [],
                       exclude = ['MEG 0312', 'EEG 005']))

```mne.pick_types()``` is different. It is actually more handy, given that you give a boolean type argument for the channel type 

In [ ]:
print(mne.pick_types(data_info, meg=False, eeg=True, exclude=[])) # select only eeg channels

#### We can select, drop or re-order channels in several ways using both of the aboove types:

In [ ]:
raw_temp = raw.copy() # create a temp raw file to play around with the channels 

# get only the eeg and eog channels
eeg_and_eog = raw_temp.pick_types(meg=False, eeg=True, eog=True) 
print(eeg_and_eog)

In [ ]:
print(len(raw.ch_names), '-', len(eeg_and_eog.ch_names)) # see differences in raw and raw_temp

In [ ]:
print('Number of channels in raw_temp:')
print(len(raw_temp.ch_names), end = ' → drop two → ')
#raw_temp.drop_channels(['EEG 037', 'EEG 059'])
print(len(raw_temp.ch_names), end=' → pick three → ')
raw_temp.pick_channels(['MEG 1811', 'EEG 017', 'EOG 061'])
print(len(raw_temp.ch_names))

### More ways to extract information from the Raw object

Use the ```Raw.info``` atribbute. This is very much like the the Info object from above. To extract information from the raw obejct we just use: ```raw.``` and the attribute we need:

In [ ]:
ntime_samples = raw.n_times 
print(ntime_samples)

time_insecs = raw.times
print(time_insecs)

time_insecs.shape # look at the shape of the time attribute. This shows the time points 

ch_names = raw.ch_names
n_chan = len(ch_names) # to look at the number of chnnels 
print(n_chan)

print('the (cropped) sample data object has {} time samples and {} channels.'
      ''.format(ntime_samples, n_chan))

print('The last time sample is at {} seconds.'.format(time_insecs[-1]))
print('The first few channel names are {}.'.format(', '.join(ch_names[:3])))
print()  # insert a blank line in the output

In [ ]:
# some examples of raw.info attribute
print('bad channels:', raw.info['bads']) # channels that are considered bad
print(raw.info['sfreq'], 'Hz') # sampling freq
print(raw.info['description']) # misc acquisition info

In [ ]:
# or look at the .info atribute:
print(raw.info)

# or..

print(raw.info.keys()) # to print only the keys (given that the raw type object is like a dict)

### Time and sample number 

One method of Raw object that is frequently used is the ```time_as_index()``` , which converts a time (in seconds) into the integer index of the sample occurring closest to that time.

It is important to remember that there may not be a data sample at *exactly* the time requested, so the number of samples between ``time = 1`` second and ``time = 2`` seconds may be different than the number of samples between ``time = 2`` and ``time = 3``:

In [ ]:
print(raw.time_as_index(20))
print(raw.time_as_index([20, 30, 40]), '\n')

print(np.diff(raw.time_as_index([1, 2, 3])))

### The raw object 

Raw obejct behaves as a .np array of shape (n_channels, n_timepoints), however ```len()``` with Raw behaves differently. It returns only the timepoints 

In [ ]:
print(len(raw))

# to get the channels:
print(len(raw.ch_names))

### Working with the time domain 

We can crop the raw object if we want to limit its time domain. I cropped the raw object in the beggining to 60sec using the ```crop()``` method, however crop uses many parameteres... 

In [ ]:
raw_selected = raw.copy().crop(tmin=10, tmax=12.5)
print(raw_selected)

I used the copy method to create. a new instance of the already cropped (to 60sec) raw object and then cropped it to 2.5 sec (from 10 to 12.5 secs)

In [ ]:
print(raw_selected.times.min(), raw_selected.times.max())
raw_selected.crop(tmin=1)
print(raw_selected.times.min(), raw_selected.times.max())

##### we can also combine or concatinate seperate parts of the raw object 

In [ ]:
raw_selected1 = raw.copy().crop(tmin=30, tmax= 30.2) # 0.2 sec
raw_selected2 = raw.copy().crop(tmin=40, tmax= 41.2) # 1.2 sec
raw_selected3 = raw.copy().crop(tmin=50, tmax= 51.3) # 1.3 sec
# combine the above parts
raw_selected1.append([raw_selected2, raw_selected3]) # 2.7 sec in total 
print(raw_selected1.times.min(), raw_selected1.times.max())

### Extract and plot data from the Raw object
Extracting sections from the Raw object into a .np array for analysis or plotting:

In [ ]:
sampling_freq = raw.info['sfreq']
start_stop_sec = np.array([11, 13])
start_sample, stop_sample = (start_stop_sec * sampling_freq).astype(int)
channel_index = 0
raw_selection = raw[channel_index, start_sample:stop_sample]

print(raw_selection) # this is a tupple containing 2 arrays 

In [ ]:
# now plot the two arrays of the tupple as x and y
x = raw_selection[1]
y = raw_selection[0].T 
plt.plot(x,y)

### Events 

Events are stored in trigger channel or channels. We can look at the events using the ```mne.find_events()``` method

In [ ]:
events = mne.find_events(raw, stim_channel ='STI 014')

### Read events and save to a file

events are better be stored in an np array, that way they can be easily saved as .npy files

In [ ]:
sample_data_events_file = os.path.join(sample_data_folder, 'MEG', 'sample',
                                       'sample_audvis_raw-eve.fif')
events_from_file = mne.read_events(sample_data_events_file)
assert np.array_equal(events, events_from_file[:len(events)])

In [ ]:
print(events_from_file.shape) # in raw datafile the shape of the events file is 320x3

Now are events are stored in an npy file that can be easily manipulated, reshaped and so on..

#### subselect and combine events 

There are several functions to manipulate events such as: ```pick_events()```, ```read_events```, and they have **include** and **exclude** parameters just like the in channel manipulation. I'll use the ```pick_events()``` method to subselect events not needed. The event type 32 for example (corresponds to buttonpress)

In [ ]:
events_withoutbutton = mne.pick_events(events, exclude = 32)

# now merge the event IDs 1, 2, 3 into one event labelad as 1:
merged_events = mne.merge_events(events, [1, 2, 3], 1)
print(np.unique(merged_events[:, -1]))

### Now I need to map the events to trial types 

Which events type (trigger nb) corresponds to which trial type orexperimental condition? This experiment had 5 conditions. 

We used these dictionaries for epoch extraction from the continuous data. 

In [ ]:
# Create events dict with the key-conditions and their corresponding values:
event_dict = {'auditory/left': 1, 'auditory/right': 2, 'visual/left': 3,
              'visual/right': 4, 'smiley': 5, 'buttonpress': 32}

# check the dict
print(event_dict.keys())
print(event_dict.values())

The ```/``` that separates two keys is used as **partial condition descriptor**. For example if we request the auditory event, we get two IDs (1, 2 for left and right)

Let's plot events to vidualise their frequency, duration and do make sure that all is as expected

In [ ]:
events_fig = mne.viz.plot_events(events, event_id = event_dict, sfreq = None,
                                first_samp = raw.first_samp)
'''
events_fig = mne.viz.plot_events(events, event_id = event_dict, sfreq = raw.info['sfreq'],
                                first_samp = raw.first_samp)'''

The ```sfreq``` parameter refers to the sample frequency. If this is none, then data is displayed in samples (not seconds). 
The parameter ```first_samp```is the index of the first sample. Recordings made in a MEG neuromag system count samples relative to the system start not to the start of the recording. In such cases the ```raw.first_samp``` is better to be passsed here. Default is zero.

#### Plot events and raw data together 

In [ ]:
# use the raw.plot method to plot events and eeg raw data together 
raw.plot(events=events, start=5, duration = 10, color='gray',
        event_color = {1:'r', 2: 'g', 3: 'b', 4: 'm', 5: 'y', 32: 'k'})

### Annotate the continuous data

Anotations are a way of storing strings of info about the temporal spans of the Raw object. They are list-like obejcts with three pieces: 
* onset time (in secs)
* duration (in secs)
* description (text)
* orig_time (time of the first sample)

Here, I won't add orig_time (leave it as None) when creating the annotations list, and by default it is assumed that orig_time is the first sample element in the raw obejct. 

I will add it afterwards:

In [ ]:
annotations = mne.Annotations(onset=[3,5,7],
                             duration = [1, 0.5, 0.25],
                             description = ['aaa', 'bbb', 'ccc'])

print(annotations)

raw.set_annotations(annotations)
print(raw.annotations)

# now convert date (a tuple of secs and microsecs) into float and add as orig_time
meas_date = raw.info['meas_date']
orig_time = raw.annotations.orig_time

print(orig_time)
print(meas_date)
print(meas_date == orig_time)

### Plot the raw data in several ways

interacting data browsing with raw.plot()

In [ ]:
raw.plot() # use left, right, up and down keys to inspect the recording

In [ ]:
# plot sepctral density of the continuous data (frequency content of the data)
raw.plot_psd(average=True)

In [ ]:
# or plot Psd for every sensor 
raw.plot_psd_topo()